In [3]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, date
import csv
import time
import ssl
import os
import threading
ssl._create_default_https_context = ssl._create_unverified_context
pd.options.mode.chained_assignment = None  # default='warn'

In [181]:
DEBUG = False
otc_list_path = r".\db\otc.csv"
twse_list_path = r".\db\twse.csv"
db_path = r"D:\Stock\db"
end_date = "113/11/01" # None

In [182]:
# time_data = [2024, 11, 5]
# output_file_path = r"D:\Stock\daily_result\%s%s%s_trend.csv"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))
# output_file_path = r"D:\Stock\daily_result\%s%s%s_start.csv"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))

In [183]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

def print_log(ss, is_debug):
    if is_debug:
        print(ss)

In [184]:
def get_twse_stock_info(df, stock):
    target_data = df[df["股票代號"] == int(stock)]
    name = target_data.iloc[0]['股票名稱']
    priceEarningRatio = target_data.iloc[0]['本益比']
    yieldRatio = target_data.iloc[0]['殖利率(%)']
    priceBookRatio = target_data.iloc[0]['股價淨值比']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_otc_stock_info(df, stock):
    target_data = df[df['SecuritiesCompanyCode'] == stock]
    name = target_data.iloc[0]['CompanyName']
    priceEarningRatio = target_data.iloc[0]['PriceEarningRatio']
    dividendPerShare = target_data.iloc[0]['DividendPerShare']
    yieldRatio = target_data.iloc[0]['YieldRatio']
    priceBookRatio = target_data.iloc[0]['PriceBookRatio']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_twse_stock_db_info():
    link = 'http://www.twse.com.tw/exchangeReport/BWIBBU_ALL?response=open_data'
    df = pd.read_csv(link, encoding='utf_8_sig')
    return df

def get_otc_stock_db_info():
    link = 'http://www.tpex.org.tw/openapi/v1/tpex_mainboard_peratio_analysis'
    json_data = requests.get(link).json()
    df = pd.DataFrame.from_records(json_data)
    return df

In [185]:
def calculate_target_trend(vol_data, pri_data, pri_max_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -7
    bb = -28
    if len(vol_data[bb:aa]) == 0 or len(pri_data[bb:aa]) == 0:
        return False
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    smooth_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:]) > 450
        
        if not vol_valid:
            print_log("vol invalid", DEBUG)
            return False
        
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        # if pri_5ma[-1] == 0 or pri_10ma[-1] == 0 or pri_20ma[-1] == 0 or pri_60ma[-1] == 0:
        #     return False
        
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.03 and pri_cur / pri_5ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.983
        print_log("pri_cur / pri_5ma[-1]: %f"%(pri_cur / pri_5ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.03 and pri_cur / pri_10ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.989
        print_log("pri_cur / pri_10ma[-1]: %f"%(pri_cur / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.04 and pri_5ma[-1] / pri_10ma[-1] >= 0.975
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.974
        pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.98
        print_log("pri_5ma[-1] / pri_10ma[-1]: %f"%(pri_5ma[-1] / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.06 and pri_10ma[-1] / pri_20ma[-1] >= 0.98
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.045 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.09 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        print_log("pri_10ma[-1] / pri_20ma[-1]: %f"%(pri_10ma[-1] / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.07 and pri_cur / pri_20ma[-1] >= 0.99
        pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.12 and pri_cur / pri_20ma[-1] >= 0.995
        print_log("pri_cur / pri_20ma[-1]: %f"%(pri_cur / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.04
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.022
        pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.027
        print_log("pri_10ma[-1] / pri_60ma[-1]: %f"%(pri_10ma[-1] / pri_60ma[-1]), DEBUG and not pos_valid)
        
        if not pos_valid:
            print_log("pos invalid", DEBUG)
            return False
        
        trend_valid = trend_valid and pri_10ma[-1] / pri_10ma[-5] >= 0.985
        print_log("pri_10ma[-1] / pri_10ma[-5]: %f"%(pri_10ma[-1] / pri_10ma[-5]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.985
        trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.992
        print_log("pri_20ma[-1] / pri_20ma[-8]: %f"%(pri_20ma[-1] / pri_20ma[-8]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.035
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.023
        trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.021
        print_log("pri_60ma[-1] / pri_60ma[-10]: %f"%(pri_60ma[-1] / pri_60ma[-10]), DEBUG and not trend_valid)
        
        if not trend_valid:
            print_log("trend invalid", DEBUG)
            return False
        
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-2] > 0.955
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-8:].max() > 0.96
        smooth_valid = smooth_valid and pri_data[-1] / pri_max_data[-9:].max() > 0.94
        print_log("pri_data[-1] / pri_max_data[-10:]: %f"%(pri_data[-1] / pri_max_data[-8:].max()), DEBUG and not smooth_valid)
        
        if not smooth_valid:
            print_log("smooth invalid", DEBUG)
            return False
        
        # print(pri_cur / pri_5ma[-1], pri_cur / pri_10ma[-1], pri_5ma[-1] / pri_10ma[-1], pri_10ma[-1] / pri_20ma[-1], pri_cur / pri_20ma[-1] ,pri_10ma[-1] / pri_60ma[-1])
        # print(pri_20ma[-1] / pri_20ma[-8], pri_60ma[-1] / pri_60ma[-10])
    except:
        # print("?")
        return False
    # print(vol_valid , pos_valid , trend_valid)
    return vol_valid and pos_valid and trend_valid and smooth_valid

def calculate_target_bear_trend(vol_data, pri_data, pri_max_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -7
    bb = -28
    if len(vol_data[bb:aa]) == 0 or len(pri_data[bb:aa]) == 0:
        return False
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    smooth_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:]) > 450
        
        if not vol_valid:
            print_log("vol invalid", DEBUG)
            return False
        
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        # if pri_5ma[-1] == 0 or pri_10ma[-1] == 0 or pri_20ma[-1] == 0 or pri_60ma[-1] == 0:
        #     return False
        
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.03 and pri_cur / pri_5ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.017 and pri_cur / pri_5ma[-1] >= 0.96
        print_log("pri_cur / pri_5ma[-1]: %f"%(pri_cur / pri_5ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.03 and pri_cur / pri_10ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.011 and pri_cur / pri_10ma[-1] >= 0.96
        print_log("pri_cur / pri_10ma[-1]: %f"%(pri_cur / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.04 and pri_5ma[-1] / pri_10ma[-1] >= 0.975
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.974
        pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.02 and pri_5ma[-1] / pri_10ma[-1] >= 0.968
        print_log("pri_5ma[-1] / pri_10ma[-1]: %f"%(pri_5ma[-1] / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.06 and pri_10ma[-1] / pri_20ma[-1] >= 0.98
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.045 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.026 and pri_10ma[-1] / pri_20ma[-1] >= 0.91
        print_log("pri_10ma[-1] / pri_20ma[-1]: %f"%(pri_10ma[-1] / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.07 and pri_cur / pri_20ma[-1] >= 0.99
        pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.005 and pri_cur / pri_20ma[-1] >= 0.88
        print_log("pri_cur / pri_20ma[-1]: %f"%(pri_cur / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.04
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.022
        pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] <= 0.913
        print_log("pri_10ma[-1] / pri_60ma[-1]: %f"%(pri_10ma[-1] / pri_60ma[-1]), DEBUG and not pos_valid)
        
        if not pos_valid:
            print_log("pos invalid", DEBUG)
            return False
        
        trend_valid = trend_valid and pri_10ma[-1] / pri_10ma[-5] <= 1.015
        print_log("pri_10ma[-1] / pri_10ma[-5]: %f"%(pri_10ma[-1] / pri_10ma[-5]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.985
        trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] <= 1.008
        print_log("pri_20ma[-1] / pri_20ma[-8]: %f"%(pri_20ma[-1] / pri_20ma[-8]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.035
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.023
        trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] <= 0.919
        print_log("pri_60ma[-1] / pri_60ma[-10]: %f"%(pri_60ma[-1] / pri_60ma[-10]), DEBUG and not trend_valid)
        
        if not trend_valid:
            print_log("trend invalid", DEBUG)
            return False
        
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-2] < 1.045
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-8:].max() < 1.04
        smooth_valid = smooth_valid and pri_data[-1] / pri_max_data[-9:].max() < 1.06
        print_log("pri_data[-1] / pri_max_data[-10:]: %f"%(pri_data[-1] / pri_max_data[-8:].max()), DEBUG and not smooth_valid)
        
        if not smooth_valid:
            print_log("smooth invalid", DEBUG)
            return False
        
        # print(pri_cur / pri_5ma[-1], pri_cur / pri_10ma[-1], pri_5ma[-1] / pri_10ma[-1], pri_10ma[-1] / pri_20ma[-1], pri_cur / pri_20ma[-1] ,pri_10ma[-1] / pri_60ma[-1])
        # print(pri_20ma[-1] / pri_20ma[-8], pri_60ma[-1] / pri_60ma[-10])
    except:
        # print("?")
        return False
    # print(vol_valid , pos_valid , trend_valid)
    return vol_valid and pos_valid and trend_valid and smooth_valid

In [186]:
def get_stock_history(stock, start_date, end_date, duration):
    stock_info_file_path = os.path.join(db_path, "%s.csv"%(stock))
    if not os.path.isfile(stock_info_file_path):
        return None
    df_all = pd.read_csv(stock_info_file_path)
    if start_date is not None:
        df_all = df_all[df_all["日期"] >= start_date]
        df_all = df_all.iloc[0:duration]
    elif end_date is not None:
        df_all = df_all[df_all["日期"] <= end_date]
        df_all = df_all.iloc[-duration:]
    else:
        df_all = df_all.iloc[-duration:]
    return df_all
# get_stock_history(2357, start_date=None, end_date="113/10/07", duration=80)

In [189]:
def analyze_stock(isOtc, target_list, thread_total = 1, thread_idx = 0):
    if isOtc:
        df = get_otc_stock_db_info()
        # data_path = r".\db\otc.csv"
        data_path = otc_list_path
    else:
        df = get_twse_stock_db_info()
        # data_path = r".\db\twse.csv"
        data_path = twse_list_path

    stock_list = []
    with open(data_path, newline='', encoding='utf_8_sig') as csvfile:
        line_list = csv.reader(csvfile)
        for line in line_list:
            stock_list.append(line[0])
    
    stock_list_len = len(stock_list)
    thread_item_num = np.ceil(stock_list_len / thread_total)
    idx_start = thread_item_num * thread_idx
    idx_end = idx_start + thread_item_num
    if thread_idx == thread_total - 1: # last thread
        idx_end = stock_list_len
    
    # target_list = []

    for stock in stock_list[int(idx_start):int(idx_end)]:
        # vol_data, pri_data, pri_max_data, df_all = get_time_duration_stock_info(time_data, stock, min_volumn=150, ma_num=1, isOtc=isOtc)
        
        # get stock history
        # stock_info_file_path = os.path.join(db_path, "%s.csv"%(stock))
        # if not os.path.isfile(stock_info_file_path):
        #     continue
        # df_all = pd.read_csv(stock_info_file_path)
        
        df_all = get_stock_history(stock, start_date=None, end_date=end_date, duration=100)
        # print(df_all.head)
        # break
        
        vol_data = df_all['成交張數'].values
        pri_data = df_all['收盤價'].values
        pri_max_data = df_all['最高價'].values
        pri_min_data = df_all['最低價'].values
        pri_sta_data = df_all['開盤價'].values
        
        if vol_data is None or pri_data is None:
            continue
        valid = calculate_target_trend(vol_data, pri_data, pri_max_data)
        # valid = calculate_target_bear_trend(vol_data, pri_data, pri_max_data)
        if valid:
            if isOtc:
                name, priceEarningRatio, yieldRatio, priceBookRatio = get_otc_stock_info(df, stock)
                target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], pri_max_data[-1], pri_min_data[-1], pri_sta_data[-1], 1])
                print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 1)
            else:
                name, priceEarningRatio, yieldRatio, priceBookRatio = get_twse_stock_info(df, stock)
                target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], pri_max_data[-1], pri_min_data[-1], pri_sta_data[-1], 0])
                print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 0)
        # break

In [190]:
target_list_twse = []
analyze_stock(isOtc=True, target_list=target_list_twse, thread_total = 1, thread_idx = 0)

3289 宜特 27.39 2.28 3.50 193.5 1
3455 由田 23.88 3.47 3.54 140.5 1
5210 寶碩 N/A 0.00 5.83 32.55 1
6220 岳豐 N/A 2.89 1.32 27.1 1
6727 亞泰金屬 13.22 2.53 1.70 86.5 1


In [191]:
target_list_otc = []
analyze_stock(isOtc=False, target_list=target_list_otc, thread_total = 1, thread_idx = 0)

1464 得力 34.18 1.49 1.3 16.6 0
1736 喬山 43.83 0.59 5.32 157.5 0
2357 華碩 14.29 2.74 1.75 583.0 0
2360 致茂 42.71 1.49 8.61 425.0 0
2603 長榮 6.47 4.57 0.95 213.0 0
3062 建漢 nan nan 2.48 35.7 0
3714 富采 nan 1.95 0.77 47.05 0
4536 拓凱 13.95 3.95 2.36 229.5 0
4935 茂林-KY 17.41 2.27 1.02 75.7 0
6515 穎崴 79.67 0.79 12.68 1300.0 0
9935 慶豐富 20.06 1.45 2.47 34.85 0
9938 百和 21.81 1.3 1.86 80.1 0


In [192]:
target_list = target_list_twse + target_list_otc
df_target = pd.DataFrame(target_list, columns=['Stock', 'Name', 'PE', 'Yield', 'PB', 'Price', 'Max Price', 'Min Price', 'Start Price', 'isOTC'])

In [193]:
def price_tick_refine(price):
    if price < 10: # 0.01
        price = np.round(price, 2)
    elif price < 50: # 0.05
        price = np.round(price*2, 1)/2
    elif price < 100: # 0.1
        price = np.round(price, 1)
    elif price < 500: # 0.5
        price = np.round(price*2, 0)/2
    elif price < 1000: # 1
        price = np.round(price, 0)
    else: # 5
        price = np.round(price*2, -1)/2
    return price

In [194]:
def get_takeProfit_stopLoss(price, min_price, max_price, isLong):
    takeProfit_ratio = 6
    stopLoss_ratio = 3
    buy_in_up_ratio = 2
    buy_in_down_ratio = 1.5
    if isLong:
        takeProfit_price = price * (1. + takeProfit_ratio/100)
        stopLoss_price = price * (1. - stopLoss_ratio/100)
        buy_in_up_price = price * (1. + buy_in_up_ratio/100)
        buy_in_down_price = price * (1. - buy_in_down_ratio/100)
    else:
        takeProfit_price = price * (1. - takeProfit_ratio/100)
        stopLoss_price = price * (1. + stopLoss_ratio/100)
        buy_in_up_price = price * (1. + buy_in_down_ratio/100)
        buy_in_down_price = price * (1. - buy_in_up_ratio/100)
    return price_tick_refine(takeProfit_price), price_tick_refine(stopLoss_price), price_tick_refine(buy_in_up_price), price_tick_refine(buy_in_down_price)

In [195]:
target_final_list = []
for target in target_list:
    stock = target[0]
    price = target[5]
    max_price = target[6]
    min_price = target[7]
    takeProfit_price, stopLoss_price, buy_in_up_price, buy_in_down_price = get_takeProfit_stopLoss(price, min_price, max_price, True)
    # print(target[5], takeProfit_price, stopLoss_price)
    target_final_list.append([end_date, stock, takeProfit_price, stopLoss_price, buy_in_up_price, buy_in_down_price])

df_target = pd.DataFrame(target_final_list, columns=['Date', 'Stock', 'Take Profit', 'Stop Loss', 'Buy In Up', 'Buy In Down'])

In [196]:
df_target.to_csv(r"D:\Stock\target\target.csv", encoding='utf-8-sig')

In [197]:
# THREAD_NUM = 6
# threads = []
# target_list = []
# for i in range(THREAD_NUM):
#     threads.append(threading.Thread(target = analyze_stock, args = (False, target_list, THREAD_NUM, i)))
#     threads[i].start()
    
# for i in range(THREAD_NUM):
#   threads[i].join()

# print("Done.")

In [152]:
# THREAD_NUM = 6
# threads = []
# target_list_otc = []
# for i in range(THREAD_NUM):
#     threads.append(threading.Thread(target = analyze_stock, args = (True, target_list_otc, THREAD_NUM, i)))
#     threads[i].start()
    
# for i in range(THREAD_NUM):
#   threads[i].join()

# print("Done.")